<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/RAG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU


%pip install langchain_community -qU
%pip install langchainhub -qU
%pip install langchain-chromadb -qU
%pip install pypdf -qU


# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic




# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

In [15]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

documents = PyPDFDirectoryLoader("/content/data/").load()

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

In [21]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY))
retriever=vectorstore.as_retriever()


In [31]:
# Definir el prompt de contexto
contextualize_q_system_prompt = """Dado un historial de conversación y la última pregunta del usuario
que podría hacer referencia al contexto de la conversación, reformule la última pregunta de forma
independiente para que sea una consulta de búsqueda independiente. NO responda a la pregunta,
solo reformúlela si es necesario."""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    ("human", "{question}")
])

# Definir el prompt para la generación de respuestas
qa_system_prompt = """Eres un asistente para tareas de búsqueda de respuestas.
Usa los siguientes fragmentos de contexto para responder la pregunta.
Si no sabes la respuesta, simplemente di que no lo sabes.
Mantén la respuesta concisa y directa.

{context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    ("human", "{question}")
])


In [32]:
llm = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=0.7)


In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Función para formatear los documentos recuperados
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Construir la cadena RAG usando LCEL
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | qa_prompt
    | llm
    | StrOutputParser()
)

# Función para hacer preguntas
def hacer_pregunta(pregunta):
    return rag_chain.invoke(pregunta)

# Ejemplo de uso
pregunta = "Quien es Gerard Bentley ?"
respuesta = hacer_pregunta(pregunta)
display(Markdown(respuesta))

# Ejemplo de seguimiento de conversación
def conversacion_rag():
    historial = []
    while True:
        pregunta = input("Haz una pregunta (o escribe 'salir' para terminar): ")

        if pregunta.lower() == 'salir':
            break

        respuesta

Gerard Bentley es un creador de Streamlit y ingeniero de software que actualmente trabaja en servicios web backend en una startup llamada Sensible Weather, que ofrece garantías de seguros contra mal tiempo.